In [1]:
import os
from pathlib import Path

import sys
sys.path.append("../")
from src.utils import get_device
from going_modular import data_setup, engine

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from typing import List, Tuple

import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchinfo import summary

from tqdm.notebook import tqdm

print(torch.__version__)

2.0.0


Setup device

In [2]:
device = get_device()
device

'mps'

Set hyperparameters

In [3]:
BATCH_SIZE = 32
NUM_WORKERS = 0 # MPS cannot work with more than 0
EPOCHS = 3
LR = 0.001
SEED = 42

Set dataloaders

In [4]:
data_path = Path("data/")
train_dir = data_path / "pizza_steak_sushi/train"
test_dir = data_path / "pizza_steak_sushi/test"

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
auto_transforms = weights.transforms()
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    transform = auto_transforms,
    batch_size = BATCH_SIZE
)
class_names

['pizza', 'steak', 'sushi']

Setup model

In [5]:
model = torchvision.models.efficientnet_b0(weights = weights)

### Freezing base model and changing the output layer to suit our needs
for param in model.features.parameters():
    param.requires_grad = False

torch.manual_seed(SEED)
torch.mps.manual_seed(SEED)

### Update the classifier head of our model
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True), 
    torch.nn.Linear(in_features=1280, 
                    out_features=len(class_names), # same number of output units as our number of classes
                    bias=True))

summary(model = model,
        input_size = (1, 3, 224, 224),
        col_names = ["input_size","output_size","num_params","trainable"],
        col_width = 20,
        row_settings = ["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

Setup loss functions and optimizer

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = LR)

In [7]:
from torch.utils.tensorboard import SumamryWriter
writer= SummaryWriter()
writer

ImportError: cannot import name 'SumamryWriter' from 'torch.utils.tensorboard' (/opt/homebrew/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py)